In [1]:
using Pkg
Pkg.instantiate()
Pkg.precompile()

Precompiling project...
┌ Info: Precompiling LimberJack [6b86205d-155a-4b14-b82d-b6a149ea78f2]
└ @ Base loading.jl:1260
ERROR: LoadError: LoadError: syntax: invalid character "−" near column 22
Stacktrace:
 [1] top-level scope at /home/jaimerz/PhD/LimberJack.jl/src/core.jl:123
 [2] include(::Module, ::String) at ./Base.jl:377
 [3] include(::String) at /home/jaimerz/PhD/LimberJack.jl/src/LimberJack.jl:1
 [4] top-level scope at /home/jaimerz/PhD/LimberJack.jl/src/LimberJack.jl:12
 [5] include(::Module, ::String) at ./Base.jl:377
 [6] top-level scope at none:2
 [7] eval at ./boot.jl:331 [inlined]
 [8] eval(::Expr) at ./client.jl:449
 [9] top-level scope at ./none:3
in expression starting at /home/jaimerz/PhD/LimberJack.jl/src/core.jl:123
in expression starting at /home/jaimerz/PhD/LimberJack.jl/src/LimberJack.jl:12


In [2]:
using Interpolations
using QuadGK
using OrdinaryDiffEq

In [3]:
struct CosmoPar{T}
    Ωm::T
    Ωb::T
    h::T
    n_s::T
    σ8::T
end

struct Cosmology
    cosmo::CosmoPar
    # Power spectrum
    ks::Array
    pk0::Array
    dlogk
    lplk::AbstractInterpolation
    # Redshift and background
    zs::Array
    chis::Array
    chi::AbstractInterpolation
    z_of_chi::AbstractInterpolation
    chi_max
    Dzs::Array
    Dz::AbstractInterpolation
end

Cosmology(cpar::CosmoPar; nk=256, nz=256) = begin
    # Compute linear power spectrum at z=0.
    ks = 10 .^ range(-4., stop=2., length=nk)
    dlogk = log(ks[2]/ks[1])
    tk = TkBBKS(cpar, ks)
    pk0 = @. ks^cpar.n_s * tk
    σ8_2_here = _σR2(ks, pk0, dlogk, 8.0/cpar.h)
    norm = cpar.σ8^2 / σ8_2_here
    pk0[:] = pk0 .* norm
    # OPT: interpolation method
    pki = LinearInterpolation(log.(ks), log.(pk0))

    # Compute redshift-distance relation
    zs = range(0., stop=3., length=nz)
    norm = CLIGHT_HMPC / cpar.h
    chis = [quadgk(z -> 1.0/_Ez(cpar, z), 0.0, zz, rtol=1E-5)[1] * norm
            for zz in zs]
    # OPT: tolerances, interpolation method
    chii = LinearInterpolation(zs, chis)
    zi = LinearInterpolation(chis, zs)

    # ODE solution for growth factor
    z_ini = 1000.0
    a_ini = 1.0/(1.0+z_ini)
    ez_ini = _Ez(cpar, z_ini)
    d0 = [a_ini^3*ez_ini, a_ini]
    a_s = reverse(@. 1.0 / (1.0 + zs))
    prob = ODEProblem(_dgrowth!, d0, (a_ini, 1.0), cpar)
    sol = solve(prob, Tsit5(), reltol=1E-6,
                abstol=1E-8, saveat=a_s)
    # OPT: interpolation (see below), ODE method, tolerances
    # Note that sol already includes some kind of interpolation,
    # so it may be possible to optimize this by just using
    # sol directly.
    s = vcat(sol.u'...)
    Dzs = reverse(s[:, 2] / s[end, 2])
    # OPT: interpolation method
    Dzi = LinearInterpolation(zs, Dzs)

    Cosmology(cpar, ks, pk0, dlogk, pki,
              collect(zs), chis, chii, zi, chis[end],
              Dzs, Dzi)
end

Cosmology(Ωc, Ωb, h, n_s, σ8; nk=256, nz=256) = begin
    Ωm = Ωc + Ωb
    cpar = CosmoPar(Ωm, Ωb, h, n_s, σ8)

    Cosmology(cpar, nk=nk, nz=nz)
end

Cosmology() = Cosmology(0.25, 0.05, 0.67, 0.96, 0.81)

Cosmology

In [4]:
function get_zeq(cosmo::CosmoPar)
    wm=cosmo.Ωm*cosmo.h^2
    tfac=2.725 / 2.7
    return 2.5*10^4*wm*tfac^(-4)
end

function get_keq(cosmo::CosmoPar)
    wm=cosmo.Ωm*cosmo.h^2
    tfac=2.725/2.7
    return 7.46*10^(-2)*wm*tfac^(-2)
end

function get_zdrag(cosmo::CosmoPar)
    wb=cosmo.Ωb*cosmo.h^2
    wm=cosmo.Ωm*cosmo.h^2
    b1 = 0.313*wm^-0.419*(1+0.607*wm*0.674)
    b2 = 0.238*wm^0.223
    return 1291*((wm^0.251)/(1+0.659*wm^0.028))*(1+b1*wb^b2)
end

function R(cosmo::CosmoPar, z)
    wb=cosmo.Ωb*cosmo.h^2
    tfac=2.725 / 2.7
    return 31.5*wb*tfac^-4*(z/10^3)^-1
end


function get_rs(cosmo::CosmoPar)
    keq = get_keq(cosmo)
    zeq = get_zeq(cosmo)
    zd = get_zdrag(cosmo)
    rs = sqrt(1+R(cosmo, zd))+sqrt(R(cosmo, zd)+R(cosmo, zeq))
    rs /= 1+sqrt(R(cosmo, zeq))
    rs = log(rs)
    rs *= 2/(3*keq)*sqrt(6/R(cosmo, zeq))
    return rs
end
    
function G(y)
    return y*(-6*sqrt(1+y)+(2+3y)*log((sqrt(1+y)+1)/(sqrt(1+y)-1)))
end

function f(k)
    s = get_rs(cosmo)    
    return (1/(1+(k*s/5.4)^4))
end
    
function T0(cosmo::CosmoPar, k, ac, bc)
    tfac = 2.725 / 2.7
    wm=cosmo.Ωm*cosmo.h^2
    keq = get_keq(cosmo)
    q = k/(13.41*keq)
    C = 14.2/ac + 386/(1+69.9*q^1.08)
    return (log(ℯ+1.8*bc*q))/(log(ℯ+1.8*bc*q)+C*q^2)
end 
    
function Tc(cosmo::CosmoPar, k)
   Wc = cosmo.Ωm-cosmo.Ωb
   s = get_rs(cosmo)
   keq = get_keq(cosmo)
   tfac = 2.725 / 2.7
   wm=cosmo.Ωm*cosmo.h^2
   q = k/(13.41*keq)
   a1 = (46.9*wm)^0.670*(1+(32.1*wm)^-0.532)
   a2 = (12.0*wm)^0.424*(1+(45.0*wm)^-0.582)
   ac = (a1^(-cosmo.Ωb/cosmo.Ωm))*(a2^(-(cosmo.Ωb/cosmo.Ωm)^3))
   b1 = 0.944*(1+(458*wm)^(-0.708))^(-1)
   b2 = (0.395*wm)^(-0.0266)
   bc = 1/(1+b1*((Wc/cosmo.Ωm)^b2-1))
   f = 1/(1+(k*s/5.4)^4)
   return f*T0(cosmo, k, 1, bc) + (1-f)*T0(cosmo, k, ac, bc)
end 

function Tb(cosmo::CosmoPar, k)
   wm=cosmo.Ωm*cosmo.h^2
   wb=cosmo.Ωb*cosmo.h^2
   s = get_rs(cosmo)
   zd = get_zdrag(cosmo)
   zeq = get_zeq(cosmo)
   keq = get_keq(cosmo)
   ksilk = 1.6*wb^0.52*wm^0.73*(1+(10.4*wm)^-0.95)
   ab = 2.07*keq*s*(1+R(cosmo, zd))^(-3/4)*G((1+zeq)/(1+zd))
   bb =  0.5+(cosmo.Ωb/cosmo.Ωm)+(3-2*cosmo.Ωb/cosmo.Ωm)sqrt((17.2*wm)^2+1)
   bnode = 8.41*(wm)^0.435
   ss = s/(1+(bnode/(k*s))^3)^(1/3)
   Tb = (T0(cosmo, k, 1, 1)/(1+(k*s/5.2)^2))+(ab/(1+(bb/(k*s))^3))*exp(-1.4*(k/ksilk)^1.4)
   return Tb*(sin(k*ss)/(k*ss))
end 

function Tk_Eis_Hu(cosmo::CosmoPar, k)
    Wc = cosmo.Ωm-cosmo.Ωb
    return (cosmo.Ωb/cosmo.Ωm)*Tb(cosmo, k)+(Wc/cosmo.Ωm)*Tc(cosmo, k)
end

Tk_Eis_Hu (generic function with 1 method)

In [5]:
cosmo = CosmoPar(0.25, 0.05, 0.67, 0.96, 0.81)

CosmoPar{Float64}(0.25, 0.05, 0.67, 0.96, 0.81)

In [6]:
Tk_Eis_Hu(cosmo, 10.0)

2.6705341498346443e-5

In [28]:
tk_mode = "BBK"

"BBK"

In [27]:
if tk_mode == "Eis"
    print("Hello")
elseif tk_mode == "BBK"
    print("Bye")
end

Hello